## Bonusový úkol č. 2 - stahování dat z webového zdroje
Vytvořte funkci **sync()**, která získá kompletní seznam produktů (tj. včetně dalších stránek) dostupných v kategorii
https://www.alza.cz/bezzrcadlovky-bez-objektivu/18863907.htm
a u každého produktu zjistí jeho aktuální cenu a stav skladu.
Funkce bude uchovávat získané informace a historii změn v relační databázi SQLLite3 obsahující dvě tabulky:  
* tabulku `products` a  
* tabulku `products_history`.

Struktura obou tabulek je shodná a obsahuje následující sloupce:  
* `id` TEXT - id produktu, např. OS072i1l1 (viz data-impression-id),  
* `url` TEXT - url produktu k kterému se vztahuje cena (pouze část path, viz ukázka na konci),  
* `title` TEXT - název produktu,  
* `price` DECIMAL - cena produktu s DPH k danému datu,   
* `stock_state` TEXT - stav skladu k danému datu,  
* `last_update` DATETIME - datum poslední změny hodnot záznamu v UTC  

Do tabulky `products_history` zkopírujte záznam z tabulky `products` ve chvíli, kdy se změnil nějaký sledovaný údaj (název, cena nebo stav skladu) a je potřeba aktualizovat data v tabulce `products`. Pozor, jedno `id` může mít více variant `url` s různou cenou. Při opětovném volání funkce **sync()** se prověří existence záznamu v `products`, prověří se shoda hodnot a vždy aktualizuje hodnota `last_update`, aby bylo zřejmé, ke kterému datu je informace platná.

**Předpokládaná náročnost**: 1 hodina

### Závislosti, načtení knihoven

V následující buňce deklarujte všechny závislosti

In [2]:
%pip install requests requests_cache bs4

import requests, requests_cache, sqlite3, random
import re, json, datetime
from bs4 import BeautifulSoup

#pro vývoj je vhodné zapnout cache (viz přednáška), pro finalní otestovaní tento řádek zakomentujte
#requests_cache.install_cache('devel') 

#nadeklarujeme si novy typ sloupce DECIMAL do sqlite3, abychom měli automatický převod mezi SQLite3 a Python
from decimal import Decimal
sqlite3.register_adapter(Decimal, lambda d: str(d))
sqlite3.register_converter("DECIMAL", lambda s: Decimal(s.decode('ascii')))

Note: you may need to restart the kernel to use updated packages.


### Deklarace funkce

V následujícím boxu definujte funkci **sync(name)** s jedním parametrem (název souboru s DB), která provede zadanou operaci. 
Pro přístup k DB lze s ohledem na složitost zadání použít přímo funkcionalitu vestavěného modulu sqlite3 (viz https://docs.python.org/2/library/sqlite3.html).

**TIP**: pro získání seznamu všech produktů lze použít endpoint https://www.alza.cz/Services/EShopService.svc/Filter

Mohlo by se také hodit: https://curl.trillworks.com/

In [17]:
# V tomto boxu pouze implementujte funkci ale nevolejte ji (pro vývoj si vytvořte vlastní buňky).
# nezapomeňte na cookies a hlavičky, jinak se Vám může zobrazit otázka "nejste robot?"

#get boxes from endpoint
def getBoxes(s):
    """
    Get boxes from endpoint

    Parameters
    ----------
    s:
        requests session

    Returns
    -------
    str:
        html with boxes from alza
    """
    headers = {
        'authority': 'www.alza.cz',
        'dnt': '1',
        'accept-language': 'cs-CZ',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36',
        'content-type': 'application/json; charset=UTF-8',
        'accept': 'application/json, text/javascript, */*; q=0.01',
        'cache-control': 'no-cache',
        'x-requested-with': 'XMLHttpRequest',
        'request-id': '|a997512e379f4357a649b54cb75c3b15.3fb610ee47e0443e',
        'origin': 'https://www.alza.cz',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'cors',
        'sec-fetch-dest': 'empty',
        'referer': 'https://www.alza.cz/bezzrcadlovky-bez-objektivu/18863907.htm',
        'cookie': '__uzma=13b1dde2-65a0-4ed1-b157-ef44695b964a; __uzmb=1602681498; __uzme=1991; lb_id=3dff50518ae596801084d72f0631899f; VZTX=2736279313; CCC=18863907; CriticalCSS=6858194; .AspNetCore.Culture=c%3Dcs-CZ%7Cuic%3Dcs-CZ; ai_user=gTrHEsE2aCBUi9cIZo0029|2020-10-14T13:18:17.264Z; _vwo_uuid_v2=D5BB76F154E337D53FC2D31085DC2FAF2|3cdf93c7b9e3c04746c668405e65a40c; __ssds=2; __ssuzjsr2=a9be0cd8e; __uzmbj2=1602681499; __uzmaj2=ca7c7cba-1334-4874-854c-8c618c4bfbe6; _gid=GA1.2.1169509407.1602681499; _gcl_au=1.1.394470555.1602681500; _fbp=fb.1.1602681501926.1641455788; db_ui=214d939e-31ac-32c8-2152-b8247fd79061; _hjid=ae43759a-6eb8-4351-8b53-c72061a366f1; db_uicd=8d4d7096-845d-9a67-60b3-6a23504dae96; SL_C_23361dd035530_KEY=b0375d591e85b0affec1d581bfed3c760cb8c56d; SL_C_23361dd035530_VID=Gk0LFsie0Hn; SL_C_23361dd035530_SID=8fTmbpNkJsm; TPL=1; PVCFLP=6; ai_session=f72wAyPMQH3r7oqKe/hcsu|1602681497653|1602684437193; __uzmdj2=1602684438; __uzmcj2=606872525505; i18next=cs-CZ; _gat=1; __uzmd=1602684439; __uzmc=844786736559; _gat_UA-948269-48=1; _dc_gtm_UA-948269-48=1; _ga=GA1.1.1545458139.1602681499; _ga_FGLGFS7LP0=GS1.1.1602684342.2.1.1602684481.17; sc/bezzrcadlovky-bez-objektivu/18863907.htm=5500',
    }

    data = '{"idCategory":18863907,"producers":"","parameters":[],"idPrefix":0,"prefixType":0,"page":1,"pageTo":4,"inStock":false,"newsOnly":false,"commodityStatusType":null,"upperDescriptionStatus":0,"branchId":-2,"sort":0,"categoryType":1,"searchTerm":"","sendProducers":false,"layout":0,"append":false,"leasingCatId":null,"yearFrom":null,"yearTo":null,"artistId":null,"minPrice":-1,"maxPrice":-1,"shouldDisplayVirtooal":false,"callFromParametrizationDialog":false,"commodityWearType":null,"scroll":5500,"hash":"#f&cst=null&cud=0&pg=1-2&prod=","counter":1}'

    r = s.post('https://www.alza.cz/Services/EShopService.svc/Filter', headers=headers, data=data)

    if(not r.ok):
        r.raise_for_status()
    else:
        return r.json()['d']['Boxes']

#get data from boxes
def getProductsDataFromBoxes(boxes):
    """
    Parameters
    ----------
    boxes: str
        html with products from alza
    
    Returns
    -------
    map
        map iterable of tuples containing data for every product in this order: (id, url, title, price, stock_state, datetime)
    """
    soup = BeautifulSoup(boxes, 'html.parser')
    products = soup.find_all("div", attrs={"class":"fb"})

    def getData(i):
        link = i.find("a", attrs={"data-impression-id":True})

        prodId = link.get('data-impression-id')
        prodTitle = link.get('data-impression-name')
        prodUrl = link.get('href')

        bottom = i.parent.parent.find("div", attrs={"class":"bottom"})
        try:
            prodPrice = re.sub(r'\D', '', bottom.find("span", attrs={"class":"c2"}).get_text()) #assuming no decimal values (reasonable in czk)
        except:
            prodPrice = 0
        prodStock = i.parent.parent.find("div", attrs={"class":"bottom"}).find("div", attrs={"class":["avl","avl extended"]}).find("span").get_text()
        return (prodId, prodUrl, prodTitle, prodPrice, prodStock, datetime.datetime.now())
    
    return map(getData, products)

def sync(dbfile='data.sqlite'):
    with sqlite3.connect(dbfile, detect_types=sqlite3.PARSE_DECLTYPES) as conn:
        c = conn.cursor()
        c.execute('''CREATE TABLE IF NOT EXISTS products
                  (id TEXT, url TEXT, title TEXT, price DECIMAL, stock_state TEXT, last_update DATETIME, PRIMARY KEY(id,url))''')
        
        c.execute('''CREATE TABLE IF NOT EXISTS products_history
                  (id TEXT, url TEXT, title TEXT, price DECIMAL, stock_state TEXT, last_update DATETIME)''')

        c.execute('''CREATE INDEX IF NOT EXISTS idx_id ON products (id)''')
        c.execute('''CREATE INDEX IF NOT EXISTS idx_idurl ON products_history (id, url)''')

        s = requests.session()
        
        #zde dopiste kod, predpokladana delka cca 50 radku
        boxes = getBoxes(s)
        data = getProductsDataFromBoxes(boxes) #(id, url, title, price, stock_state, last_update)
        for d in data:
            c.execute('''SELECT * FROM products WHERE id=? AND url=?''', (d[0], d[1]))
            row = c.fetchone()
            if(row):
                if(row[2] != d[2] or row[3] != d[3] or row[4] != d[4]):
                    c.execute('''INSERT INTO products_history VALUES (?, ?, ?, ?, ?, ?)''', row)
                    c.execute('''UPDATE products SET title=?, price=?, stock_state=?, last_update=? WHERE id=? AND url=?''', (d[2], d[3], d[4], d[5], d[0], d[1]))
                else:
                    c.execute('''UPDATE products SET last_update=? WHERE id=? AND url=?''', (d[5], d[0], d[1]))
            else:
                c.execute('''INSERT INTO products VALUES (?, ?, ?, ?, ?, ?)''', d)

        conn.commit()        
        c.close()

### Ověření korektní funkce

Na následujícím kódu lze ověřit základní funkcionalitu. Měly byste dostat stejný výstup jako je v ukázce. Protože se však stav e-shopu může měnit, uzpůsobte si eventuelně dotaz dle potřeb. Momentálně se testuje existence produktu https://www.alza.cz/sony-alpha-7ii?dq=2286288 ev. 
https://www.alza.cz/kod/OS072i1p5.

Při ověřování korektní funkce Vaší implementace bude porovnán obsah DB vytvořený Vaší funkcí s předpokládaným obsahem DB v určitou dobu a poté znovu s několika hodinovým odstupem.

In [18]:
from contextlib import closing

sync('data.sqlite')

with sqlite3.connect('data.sqlite', detect_types=sqlite3.PARSE_DECLTYPES) as conn:
    with closing(conn.cursor()) as c:
        c.execute('SELECT id, url, price FROM products WHERE id=? AND url=? AND price>20000', ('OS072i1p5','/sony-alpha-7ii?dq=2286288'))
        r = c.fetchone()
        print(r)
        assert(r != None)

        c.execute('SELECT id, url, price FROM products WHERE id=? AND price>30000', ('OF7032a',))
        r = c.fetchall()
        print(r)
        assert (len(r)>0 and '/fujifilm-x-t3?dq=5457426' in [a[1] for a in r])

print("OK")        

('OS072i1p5', '/sony-alpha-7ii?dq=2286288', Decimal('31490'))
[('OF7032a', '/fujifilm-x-t3?dq=5457426', Decimal('39990')), ('OF7032a', '/fujifilm-x-t3-telo-cerny-levne-d5754350.htm', Decimal('36990')), ('OF7032a', '/fujifilm-x-t3-telo-cerny-sleva-d5877920.htm', Decimal('33990'))]
OK


### Komentář
Do pole níže můžete vložit textový komentář týkající se tohoto úkolu. Např. jak dlouho Vám trvalo řešení, co bylo obtížné, co bylo se mělo více v rámci přenášky vysvětlit apod.

n/a